In [5]:
# import libraries
import sys

sys.path.append("./src/bcc_model/")

import itertools
import math

import pandas as pd
from blue_carbon_project import BlueCarbonProject
from cost_calculator import CostCalculator
from sequestration_credits_calculator import SequestrationCreditsCalculator

In [6]:
# Define functions:


def calculate_breakeven_cost(project, max_iterations=100, tolerance=1e-5):
    """
    Function to calculate the breakeven cost of carbon
    """
    # Start the calculation with the initial carbon price
    carbon_price = project.carbon_price

    for _iteration in range(max_iterations):
        # Update the project carbon price and initialize cost calculator
        project.carbon_price = carbon_price
        cost_calculator = CostCalculator(project)

        # Calculate NPV covering cost
        npv_covering_cost = cost_calculator.NPV_covering_cost
        credits_issued = cost_calculator.credits_issued

        # Check if the NPV covering cost is within the acceptable tolerance
        if abs(npv_covering_cost) < tolerance:
            # Retrieve additional cost estimates and summary
            cost_summary = cost_calculator.get_summary()  # Assuming this method exists
            cost_estimates = cost_calculator.get_cost_estimates()  # Assuming this method exists

            return {
                "breakeven_carbon_price": float(carbon_price),
                "cost_summary": cost_summary,
                "cost_estimates": cost_estimates,
            }

        # Ensure credits_issued is not zero to avoid division errors
        if credits_issued == 0:
            print("Error: Credits issued are zero, breakeven cost cannot be calculated.")
            return None

        # Update carbon price based on the NPV covering cost and credits issued
        carbon_price -= npv_covering_cost / credits_issued

    # If max_iterations are reached without convergence, return the last calculated price
    print("Warning: Max iterations reached without convergence.")
    return {
        "breakeven_carbon_price": float(carbon_price),
        "cost_summary": cost_calculator.get_summary(),
        "cost_estimates": cost_calculator.get_cost_estimates(),
    }

In [12]:
# 1. Import master table, base size table and base increase table
#  with information for the project calculations
data_path = "./raw_data/data_ingestion.xlsm"

# Open the excel file - tab 'master_table'
master_table = pd.read_excel(data_path, sheet_name="master_table")

# Open the excel file - tab 'base_size_table'
base_size = pd.read_excel(data_path, sheet_name="base_size_table")

# Open the excel file - tab 'base_increase'
base_increase = pd.read_excel(data_path, sheet_name="base_increase")

master_table.head()

,country,ecosystem,activity,country_code,continent,hdi,project_size_ha,feasibility_analysis,conservation_planning_and_admin,data_collection_and_field_cost,...,ecosystem_extent,ecosystem_extent_historic,ecosystem_loss_rate,restorable_land,tier_1_emission_factor,emission_factor_AGB,emission_factor_SOC,tier_1_sequestration_rate,tier_2_sequestration_rate,other_community_cash_flow
0,United States,Mangrove,Restoration,USA,North America,1.0,500.0,100000.0,166766.666667,26666.666667,...,2.315054e+05,2.312725e+05,-0.008868,9665.780512,27.2,55.227609,58.002081,5.94,17.2349,Non-development
1,Indonesia,Mangrove,Restoration,IDN,Asia,2.0,500.0,50000.0,166766.666667,26666.666667,...,2.901690e+06,2.930508e+06,-0.003716,204536.167900,27.2,355.398394,71.876642,5.94,29.5000,Development
2,Australia,Mangrove,Restoration,AUS,Oceania,1.0,500.0,70000.0,166766.666667,26666.666667,...,1.004149e+06,1.013552e+06,-0.001498,69160.397250,27.2,197.881108,59.762089,5.94,7.2850,Non-development
3,Caribbean,Mangrove,Restoration,BHS,North America,1.0,500.0,70000.0,166766.666667,26666.666667,...,1.508391e+05,1.447886e+05,-0.007321,9842.169833,27.2,21.830982,107.946528,5.94,7.5000,Non-development
4,Kenya,Mangrove,Restoration,KEN,Africa,3.0,500.0,50000.0,166766.666667,26666.666667,...,5.412150e+04,5.408001e+04,-0.002781,1609.698100,27.2,91.779817,79.040056,5.94,19.0000,Development


In [13]:
# 2. Define different list with all the possible values for the different parameters

# Definition of lists
country_list = [
    "United States",
    "Indonesia",
    "Australia",
    "Caribbean",
    "Kenya",
    "Mexico",
    "Colombia",
    "India",
    "China",
]
ecosystem_list = ["Mangrove", "Seagrass", "Salt marsh"]
activity_type_list = ["Restoration", "Conservation"]
restoration_activity_type_list = ["Planting", "Hydrology", "Hybrid"]

# Definition of parameters
project_size_filter = ["Small", "Medium", "Large"]
price_type = ["Opex breakeven", "Market price"]

# Define project size thresholds for each ecosystem and activity type
project_size_thresholds = {
    ("Mangrove", "Conservation"): {"Small": 4000, "Medium": 20000, "Large": 40000},
    ("Mangrove", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
    ("Salt marsh", "Conservation"): {"Small": 800, "Medium": 4000, "Large": 8000},
    ("Salt marsh", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
    ("Seagrass", "Conservation"): {"Small": 400, "Medium": 2000, "Large": 4000},
    ("Seagrass", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
}

In [14]:
# 3. Create all possible combinations for the overview page
all_combinations = []

for country, ecosystem, activity, project_size, price in itertools.product(
    country_list, ecosystem_list, activity_type_list, project_size_filter, price_type
):
    # Apply restoration activity type only if the activity is "Restoration"
    if activity == "Restoration":
        for restoration_activity in restoration_activity_type_list:
            threshold = project_size_thresholds.get((ecosystem, activity), {}).get(project_size)
            all_combinations.append(
                {
                    "country": country,
                    "ecosystem": ecosystem,
                    "activity": activity,
                    "activity_type": restoration_activity,
                    "project_size_filter": project_size,
                    "project_size": threshold,
                    "price_type": price,
                }
            )
    else:
        # For Conservation, restoration activity type should be None
        threshold = project_size_thresholds.get((ecosystem, activity), {}).get(project_size)
        all_combinations.append(
            {
                "country": country,
                "ecosystem": ecosystem,
                "activity": activity,
                "activity_type": None,
                "project_size_filter": project_size,
                "project_size": threshold,
                "price_type": price,
            }
        )

# Convert to DataFrame
df_combinations = pd.DataFrame(all_combinations)

# Display the first few rows of the DataFrame
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven
1,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven
2,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven
3,United States,Mangrove,Restoration,Planting,Small,100,Market price
4,United States,Mangrove,Restoration,Hydrology,Small,100,Market price


In [15]:
# 4. Apply model constrains

# Remove all rows for Seagrass restoration projects that are not Planting
df_combinations = df_combinations[
    ~(
        (df_combinations["ecosystem"] == "Seagrass")
        & (df_combinations["activity"] == "Restoration")
        & (df_combinations["activity_type"] != "Planting")
    )
]

# Define conditions for removal based on country, ecosystem, activity, and activity_type
conditions_to_remove = [
    # List of country-ecosystem pairs to remove 'Hybrid' restoration activities
    # Remove the projects for Indonesia Manrgove Restoaration Hybrid
    # Remove projects for Indonesia Salt Marsh Restoration Hybrid
    # Remove projects for Australia Mangrove Restoration Hybrid
    # Remove pprojects for Australia Salt Marsh Restoration Hybrid
    # Remove projects for Caribbean Mangrove Restoration Hybrid
    # Remove project for Caribbean Salt marsh Restoration Hybrid
    # Remove projects for Kenya Mangrove Restoration Hybrid
    # Remove projects for Kenya Salt marsh Restoration Hybrid
    # Remove projects for Colombia mangrove restoration hybrid
    # Remove projects for Colombia Salt marsh Restoration Hybrid
    # Remove prjects India Manrgove restoration Hybrid
    # Remove projects for India Salt marsh Restoration Hybrid
    # Remove projects for China Mangrove Restoration Hybrid
    # Remove projects for China Salt marsh Restoration Hybrid
    ("Indonesia", "Mangrove"),
    ("Indonesia", "Salt marsh"),
    ("Australia", "Mangrove"),
    ("Australia", "Salt marsh"),
    ("Caribbean", "Mangrove"),
    ("Caribbean", "Salt marsh"),
    ("Kenya", "Mangrove"),
    ("Kenya", "Salt marsh"),
    ("Colombia", "Mangrove"),
    ("Colombia", "Salt marsh"),
    ("India", "Mangrove"),
    ("India", "Salt marsh"),
    ("China", "Mangrove"),
    ("China", "Salt marsh"),
]

# 3. Apply the conditions to filter out specified rows in a single operation
df_combinations = df_combinations[
    ~(
        (df_combinations["activity"] == "Restoration")
        & (df_combinations["activity_type"] == "Hybrid")
        & (
            df_combinations[["country", "ecosystem"]]
            .apply(tuple, axis=1)
            .isin(conditions_to_remove)
        )
    )
]

In [16]:
# 5. Get the country name, country_code and continent information from the master table:

# Filter master table just for the columns we need
master_table_filtered = master_table[["country", "country_code", "continent"]]
master_table_filtered.head()

# left merge the master_table with the df_combinations
df_combinations = df_combinations.merge(master_table_filtered, how="left", on=["country"])
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_code,continent
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America
1,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America
2,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America
3,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America
4,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America


In [17]:
# 6. Get the project_size_ha from the master_table and rename it to base_size
base_size_value_df = master_table[["country_code", "ecosystem", "activity", "project_size_ha"]]

# Merge base_size_value_df with df_combinations
df_combinations = df_combinations.merge(
    base_size_value_df, how="left", on=["country_code", "ecosystem", "activity"]
)
# Rename the column project_size_ha to base_size
df_combinations.rename(columns={"project_size_ha": "base_size"}, inplace=True)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_code,continent,base_size
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America,500.0
1,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America,500.0
2,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America,500.0
3,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America,500.0
4,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America,500.0


In [18]:
# 7. Create a new column called "project_name" by concatenating the "country", "ecosystem",
# "activity" and "project_size" columns
df_combinations["project_name"] = (
    df_combinations["country"]
    + df_combinations["ecosystem"]
    + df_combinations["activity"]
    + df_combinations["project_size"].astype(str)
)
# drop duplicates in case we have some duplicated project names
df_combinations.drop_duplicates(inplace=True)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_code,continent,base_size,project_name
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America,500.0,United StatesMangroveRestoration100
6,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,USA,North America,500.0,United StatesMangroveRestoration100
12,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,USA,North America,500.0,United StatesMangroveRestoration100
18,United States,Mangrove,Restoration,Planting,Small,100,Market price,USA,North America,500.0,United StatesMangroveRestoration100
24,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,USA,North America,500.0,United StatesMangroveRestoration100


In [19]:
# 8. Run the model for each project and calculate the costs

# Step 1: Initialize columns for the output df with None
columns_to_add = [
    "total_cost_npv",
    "total_cost",
    "capex_npv",
    "capex",
    "opex_npv",
    "opex",
    "abatement_potential",
    "cost_per_tco2e",
    "cost_per_tco2e_npv",
    "feasibility_analysis_npv",
    "feasibility_analysis",
    "conservation_planning_npv",
    "conservation_planning",
    "data_collection_npv",
    "data_collection",
    "community_representation_npv",
    "community_representation",
    "blue_carbon_project_planning_npv",
    "blue_carbon_project_planning",
    "establishing_carbon_rights_npv",
    "establishing_carbon_rights",
    "validation_npv",
    "validation",
    "implementation_labor_npv",
    "implementation_labor",
    "monitoring_maintenance_npv",
    "monitoring_maintenance",
    "community_benefit_npv",
    "community_benefit",
    "carbon_standard_fees_npv",
    "carbon_standard_fees",
    "baseline_reassessment_npv",
    "baseline_reassessment",
    "mrv_npv",
    "mrv",
    "long_term_project_operating_npv",
    "long_term_project_operating",
    "initial_price_assumption",
    "leftover_after_opex",
    "leftover_after_opex_npv",
    "total_revenue",
    "total_revenu_npv",
    "credits_issued",
]
df_combinations[columns_to_add] = None

for index, row in df_combinations.iterrows():
    projet_name = row["project_name"]
    print(f"Calculating project {projet_name}...")
    # initial_price_assumption = 30

    # Set up common BlueCarbonProject parameters
    project_params = {
        "activity": row["activity"],
        "ecosystem": row["ecosystem"],
        "country": row["country"],
        "master_table": master_table,
        "base_size": base_size,
        "base_increase": base_increase,
        # 'carbon_price': initial_price_assumption,
        "carbon_revenues_to_cover": "Opex",
        "project_size_ha": row["project_size"],
        "restoration_activity": row["activity_type"],
        "planting_success_rate": 0.8,
        "loss_rate_used": "National average",
    }

    # Set ecosystem-specific rates
    if row["ecosystem"] == "Mangrove":
        project_params.update(
            {
                "sequestration_rate_used": "Tier 2 - Country-specific rate",
                "emission_factor_used": "Tier 2 - Country-specific emission factor",
            }
        )
    else:
        print(f"No default Tier 2 sequestration rate data available for {row['ecosystem']}.")
        project_params.update(
            {
                "sequestration_rate_used": "Tier 1 - IPCC default value",
                "emission_factor_used": "Tier 1 - Global emission factor",
            }
        )

    # # Initialize project
    # project = BlueCarbonProject(**project_params)

    # Determine the initial carbon price based on price type
    initial_price_assumption = 30
    project_params["carbon_price"] = initial_price_assumption

    # Initialize project with the initial or market price
    project = BlueCarbonProject(**project_params)

    if row["price_type"] == "Market price":
        df_combinations.loc[index, "initial_price_assumption"] = initial_price_assumption
        # cost_calculator = CostCalculator(project)
        # summary = cost_calculator.get_summary()
        # cost_estimates = cost_calculator.get_cost_estimates()
    elif row["price_type"] == "Opex breakeven":
        try:
            breakeven = calculate_breakeven_cost(project, max_iterations=150)
            breakeven_cost = breakeven["breakeven_carbon_price"]
            # print(f"Breakeven cost for project {row['project_name']} is {breakeven_cost}")
            df_combinations.loc[index, "initial_price_assumption"] = breakeven_cost
            # summary = breakeven['cost_summary']
            # cost_estimates = breakeven['cost_estimates']
            # Update project_params with breakeven carbon price
            project_params["carbon_price"] = breakeven_cost

            # Re-initialize project with breakeven price
            project = BlueCarbonProject(**project_params)
        except:  # noqa: E722
            print(f"Error calculating breakeven cost for project {row['project_name']}")
            df_combinations.loc[index, "initial_price_assumption"] = None
            # Re-initialize project with the initial price if breakeven calculation fails
            project_params["carbon_price"] = (
                initial_price_assumption  # <-- Set to initial price if error
            )
            project = BlueCarbonProject(**project_params)
            # cost_calculator = CostCalculator(project)
            # summary = cost_calculator.get_summary()
            # cost_estimates = cost_calculator.get_cost_estimates()

    # print('project params:', project_params)
    # Step 3: Calculate costs and extract data into dictionary
    cost_calculator = CostCalculator(project)
    cost_estimates = cost_calculator.get_cost_estimates()
    summary = cost_calculator.get_summary()

    # Define a dictionary for cost components mapping
    cost_mapping = {
        "total_cost_npv": ("Total cost", "NPV"),
        "total_cost": ("Total cost", "Total cost"),
        "capex_npv": ("Capital expenditure", "NPV"),
        "capex": ("Capital expenditure", "Total cost"),
        "opex_npv": ("Operating expenditure", "NPV"),
        "opex": ("Operating expenditure", "Total cost"),
        "feasibility_analysis_npv": ("Feasibility analysis", "NPV"),
        "feasibility_analysis": ("Feasibility analysis", "Total cost"),
        "conservation_planning_npv": ("Conservation planning and admin", "NPV"),
        "conservation_planning": ("Conservation planning and admin", "Total cost"),
        "data_collection_npv": ("Data collection and field costs", "NPV"),
        "data_collection": ("Data collection and field costs", "Total cost"),
        "community_representation_npv": ("Community representation / liaison", "NPV"),
        "community_representation": ("Community representation / liaison", "Total cost"),
        "blue_carbon_project_planning_npv": ("Blue carbon project planning", "NPV"),
        "blue_carbon_project_planning": ("Blue carbon project planning", "Total cost"),
        "establishing_carbon_rights_npv": ("Establishing carbon rights", "NPV"),
        "establishing_carbon_rights": ("Establishing carbon rights", "Total cost"),
        "validation_npv": ("Validation", "NPV"),
        "validation": ("Validation", "Total cost"),
        "implementation_labor_npv": ("Implementation labor", "NPV"),
        "implementation_labor": ("Implementation labor", "Total cost"),
        "monitoring_npv": ("Monitoring", "NPV"),
        "maintenance_npv": ("Maintenance", "NPV"),
        "monitoring": ("Monitoring", "Total cost"),
        "maintenance": ("Maintenance", "Total cost"),
        "community_benefit_npv": ("Community benefit sharing fund", "NPV"),
        "community_benefit": ("Community benefit sharing fund", "Total cost"),
        "carbon_standard_fees_npv": ("Carbon standard fees", "NPV"),
        "carbon_standard_fees": ("Carbon standard fees", "Total cost"),
        "baseline_reassessment_npv": ("Baseline reassessment", "NPV"),
        "baseline_reassessment": ("Baseline reassessment", "Total cost"),
        "mrv_npv": ("MRV", "NPV"),
        "mrv": ("MRV", "Total cost"),
        "long_term_project_operating_npv": ("Long-term project operating", "NPV"),
        "long_term_project_operating": ("Long-term project operating", "Total cost"),
    }

    # Loop over each cost component and assign
    for col_name, (cost_label, column) in cost_mapping.items():
        df_combinations.loc[index, col_name] = (
            cost_estimates.loc[cost_estimates["Cost estimates (USD)"] == cost_label, column]
            .values[0]
            .replace("$", "")
            .replace(",", "")
        )

    # Add monitoring_maintenance_npv
    df_combinations.loc[index, "monitoring_maintenance_npv"] = float(
        df_combinations.loc[index, "monitoring_npv"]
    ) + float(df_combinations.loc[index, "maintenance_npv"])
    df_combinations.loc[index, "monitoring_maintenance"] = float(
        df_combinations.loc[index, "monitoring"]
    ) + float(df_combinations.loc[index, "maintenance"])

    # Other calculations
    sequestration_credits_calculator = SequestrationCreditsCalculator(project)
    abatement_potential = round(sequestration_credits_calculator.calculate_abatement_potential())
    df_combinations.loc[index, "abatement_potential"] = abatement_potential

    # Safely handle NaN in cost_per_tCO2e
    cost_per_tCO2e = cost_calculator.cost_per_tCO2e  # noqa: N816
    df_combinations.loc[index, "cost_per_tco2e"] = (
        round(cost_per_tCO2e) if not math.isnan(cost_per_tCO2e) else 0
    )

    # Safely handle NaN in cost_per_tCO2e NPV
    cost_per_tCO2e_npv = cost_calculator.cost_per_tCO2e  # noqa: N816
    df_combinations.loc[index, "cost_per_tco2e"] = (
        round(cost_per_tCO2e) if not math.isnan(cost_per_tCO2e) else 0
    )

    # Add total revenue, total revenue NPV, and credits issued
    df_combinations.loc[index, "total_revenue"] = (
        summary.get("Total revenue (non-discounted)", 0).replace("$", "").replace(",", "")
    )
    df_combinations.loc[index, "total_revenu_npv"] = (
        summary.get("Total revenue (NPV)", 0).replace("$", "").replace(",", "")
    )
    df_combinations.loc[index, "credits_issued"] = summary.get("Credits issued", 0)

    # # Add left_over_after_opex and left_over_after_opex_npv
    df_combinations["leftover_after_opex_npv"] = df_combinations["total_revenu_npv"].astype(
        float
    ) - df_combinations["opex_npv"].astype(float)
    df_combinations["leftover_after_opex"] = df_combinations["total_revenue"].astype(
        float
    ) - df_combinations["opex"].astype(float)

Calculating project United StatesMangroveRestoration100...
Calculating project United StatesMangroveRestoration100...
Calculating project United StatesMangroveRestoration100...
Calculating project United StatesMangroveRestoration100...
Calculating project United StatesMangroveRestoration100...
Calculating project United StatesMangroveRestoration100...
Calculating project United StatesMangroveRestoration500...
Calculating project United StatesMangroveRestoration500...
Calculating project United StatesMangroveRestoration500...
Calculating project United StatesMangroveRestoration500...
Calculating project United StatesMangroveRestoration500...
Calculating project United StatesMangroveRestoration500...
Calculating project United StatesMangroveRestoration1000...
Calculating project United StatesMangroveRestoration1000...
Calculating project United StatesMangroveRestoration1000...
Calculating project United StatesMangroveRestoration1000...
Calculating project United StatesMangroveRestoration

In [20]:
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_code,continent,base_size,...,initial_price_assumption,leftover_after_opex,leftover_after_opex_npv,total_revenue,total_revenu_npv,credits_issued,monitoring_npv,maintenance_npv,monitoring,maintenance
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,USA,North America,500.0,...,190.103739,191780.0,0.0,4921553,2916212,"22,061",600463,290522,994000,353284
6,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,USA,North America,500.0,...,260.495904,1122892.0,0.0,8429900,4995044,"27,576",600463,2264746,994000,2753998
12,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,USA,North America,500.0,...,251.892764,1048980.0,0.0,8151494,4830077,"27,576",600463,2108028,994000,2563424
18,United States,Mangrove,Restoration,Planting,Small,100,Market price,USA,North America,500.0,...,30,-3745866.0,-2333208.0,776663,460203,"22,061",600463,290522,994000,353284
24,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,USA,North America,500.0,...,30,-5963226.0,-4198800.0,970829,575254,"27,576",600463,2264746,994000,2753998


In [21]:
df_combinations.columns

Index(['country', 'ecosystem', 'activity', 'activity_type',
       'project_size_filter', 'project_size', 'price_type', 'country_code',
       'continent', 'base_size', 'project_name', 'total_cost_npv',
       'total_cost', 'capex_npv', 'capex', 'opex_npv', 'opex',
       'abatement_potential', 'cost_per_tco2e', 'cost_per_tco2e_npv',
       'feasibility_analysis_npv', 'feasibility_analysis',
       'conservation_planning_npv', 'conservation_planning',
       'data_collection_npv', 'data_collection',
       'community_representation_npv', 'community_representation',
       'blue_carbon_project_planning_npv', 'blue_carbon_project_planning',
       'establishing_carbon_rights_npv', 'establishing_carbon_rights',
       'validation_npv', 'validation', 'implementation_labor_npv',
       'implementation_labor', 'monitoring_maintenance_npv',
       'monitoring_maintenance', 'community_benefit_npv', 'community_benefit',
       'carbon_standard_fees_npv', 'carbon_standard_fees',
       'baseline

In [13]:
# export to csv
df_combinations.to_csv("../../raw_data/output.csv", index=False)